In [1]:
#!/usr/bin/env python
#-*- coding:utf-8 -*-

import numpy as np
import pandas as pd
import scipy.ndimage
import skimage.morphology
import sklearn.mixture


class HDoG_CPU(object):
    def __init__(self, width=2048, height=2048, depth=None, sigma_xy=(4.0, 6.0), sigma_z=(1.8,2.7),
                 radius_small=(24,3), radius_large=(100,5), min_intensity=1000, gamma=1.0):
        self.width  = width
        self.height = height
        self.depth  = depth

        if type(sigma_xy) in [float,int]:
            self.sigma_xy = (sigma_xy, sigma_xy*1.5)
        else:
            self.sigma_xy = sigma_xy
        if type(sigma_z) in [float,int]:
            self.sigma_z = (sigma_z, sigma_z*1.5)
        else:
            self.sigma_z = sigma_z

        if not radius_small:
            self.radius_small_xy = int(self.sigma_xy[1]*4)
            self.radius_small_z = int(self.sigma_z[1]*2)
        else:
            self.radius_small_xy = radius_small[0]
            self.radius_small_z = radius_small[1]
        self.size_small = (self.radius_small_z*2+1, self.radius_small_xy*2+1, self.radius_small_xy*2+1)#self.radius_small_z*2+1

        if not radius_large:
            self.radius_large_xy = int(self.sigma_xy[1]*30)
            self.radius_large_xy = int(self.sigma_z[1]*10)
        else:
            self.radius_large_xy = radius_large[0]
            self.radius_large_z = radius_large[1]
        self.size_large = (self.radius_large_z*2+1, self.radius_large_xy*2+1, self.radius_large_xy*2+1)#self.radius_large_z*2+1

        self.min_intensity = min_intensity

        self.gamma = gamma
        self.normalizer = (self.sigma_xy[0]**(gamma*2)) * (self.sigma_z[0]**gamma)

    def load_images(self, list_images, dtype=np.uint16, height=2048, width=2060):
        # self.height, self.width, self.depthを書き換えた
        imgs = []
        for path in list_images:
            img = np.fromfile(path, dtype=dtype).reshape(height, width)
            imgs.append(img)
        imgs = np.array(imgs)
        depth = imgs.shape[0]
        return imgs

    def Normalize(self, src_img):
        print("Normalize start")
        dilation_l_img = scipy.ndimage.filters.uniform_filter(
            scipy.ndimage.morphology.grey_dilation(src_img, size=size_large, mode="nearest").astype(np.float32),
            size=size_large, mode="constant", cval=0)
        print("Dilation finish")
        erosion_l_img = scipy.ndimage.filters.uniform_filter(
            scipy.ndimage.morphology.grey_erosion(src_img, size=size_large, mode="nearest").astype(np.float32),
            size=size_large, mode="constant", cval=0)
        print("Erosion finish")

        intensity = src_img.astype(np.float32)
        #norm_img = ((intensity >= min_intensity) * intensity - erosion_l_img) / (dilation_l_img - erosion_l_img)
        norm_img = (intensity >= min_intensity) * intensity / (dilation_l_img - erosion_l_img)
        print("Normalize finish")
        return norm_img

    def DoGFilter(self, src_img):
        temp1 = scipy.ndimage.filters.gaussian_filter(
            src_img.astype(np.float32),
            sigma=(sigma_z[0],sigma_xy[0],sigma_xy[0]), #sigma_z[0]
            truncate=2.0, mode="constant", cval=0)
        temp2 = scipy.ndimage.filters.gaussian_filter(
            src_img.astype(np.float32),
            sigma=(sigma_z[1],sigma_xy[1],sigma_xy[1]), #sigma_z[1]
            truncate=2.0, mode="constant", cval=0)
        dog_img = (temp1 - temp2) * normalizer
        return dog_img

    def HessianPDFilter(self, dog_img):
        Hz,Hy,Hx = np.gradient(dog_img)
        Hzz,Hyz,Hxz = np.gradient(Hz)
        Hyz,Hyy,Hxy = np.gradient(Hy)
        Hxz,Hxy,Hxx = np.gradient(Hx)
        det_img = Hxx*Hyy*Hzz + 2*Hxy*Hyz*Hxz - Hxx*Hyz*Hyz - Hyy*Hxz*Hxz - Hzz*Hxy*Hxy
        pd_img = np.bitwise_and(np.bitwise_and(Hxx < 0, Hxx*Hyy-Hxy*Hxy > 0), det_img < 0)
        hessian_img = np.array([Hxx,Hxy,Hxz,Hyy,Hyz,Hzz])
        return pd_img, hessian_img

    def ScaleResponse(self, scale_img, pd_img):
        response = np.sum(scale_img*pd_img) / np.sum(pd_img)
        return response

    def CCL(self, pd_img):
        labels_img = skimage.morphology.label(pd_img)
        return labels_img

In [2]:
import cv2
import numpy as np
from matplotlib import pyplot as plt
from scipy import ndimage
from skimage import measure, color, io

%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import joblib, json, sys, os
from IPython.display import Image, display_png

#sys.path.append("script/")
#from MergeBrain import WholeBrainCells
import json, glob, os.path

#import ants
import tifffile

In [3]:
# 脳と、diff後でも、threshある
    def RegionalFeatures3(self, norm_img, labels_img):
        on_region = np.nonzero(labels_img)
        labels_list = labels_img[on_region]
        num_labels = np.max(labels_list)

        # max intensity
        max_normalized = scipy.ndimage.maximum(norm_img, labels=labels_img, index=range(1, num_labels+1))

        # region size
        ns = np.ones(len(labels_list))
        region_size = np.bincount(labels_list-1, weights=ns)

        zgrid,ygrid,xgrid = np.mgrid[0:depth, 0:height, 0:width]#np.mgrid[0:self.depth, 0:self.height, 0:self.width]
        centroid_x = np.bincount(labels_img.flatten(), weights=xgrid.flatten())[1:] / region_size#np.bincount(labels_img.flatten())[1:]
        centroid_y = np.bincount(labels_img.flatten(), weights=ygrid.flatten())[1:] / region_size#np.bincount(labels_img.flatten())[1:]
        centroid_z = np.bincount(labels_img.flatten(), weights=zgrid.flatten())[1:] / region_size#np.bincount(labels_img.flatten())[1:]

        df = pd.DataFrame({
            "index": pd.Series(np.arange(num_labels)),
            "intensity": pd.Series(max_normalized),
            "size": pd.Series(region_size),
            "centroid_x": pd.Series(centroid_x),
            "centroid_y": pd.Series(centroid_y),
            "centroid_z": pd.Series(centroid_z),
        })
        return df

    def classify_unsupervised(self, features, i_feature_maximize=2, n_components=3):
        # input
        # features : (num_regions, num_features)
        # i_feature_maximize : the cluster is selected as desired one
        #                      if it has maximum mean value of this feature
        # n_components : number of gaussians
        #
        # output
        # pred : a region is blob if pred = 1, not blob if pred = 0

        vbgmm = sklearn.mixture.BayesianGaussianMixture(weight_concentration_prior_type="dirichlet_process", n_components=n_components)
        vbgmm.fit(features)
        pred = vbgmm.predict(features)

        i_pred_cluster = np.argmax(vbgmm.means_[:,i_feature_maximize])
        return pred == i_pred_cluster

In [4]:
#width=cells16.shape[0]
#height=cells16.shape[1]
depth=3
#sigma_xy=(4.0, 6.0)
#sigma_z=(1.8,2.7)
sigma_z=(1, 1)
#sigma_xy=(4.0, 6.0)
sigma_xy=(8.0, 10.0)
#sigma_z=(1,1)
#sigma_z=(0.5,0.5)
radius_small=(24,3)
radius_large=(100,5)
#min_intensity=63 #25
gamma=1.0

In [5]:
        if type(sigma_xy) in [float,int]:
            sigma_xy = (sigma_xy, sigma_xy*1.5)
        else:
            sigma_xy = sigma_xy
        if type(sigma_z) in [float,int]:
            sigma_z = (sigma_z, sigma_z*1.5)
        else:
            sigma_z = sigma_z

        if not radius_small:
            radius_small_xy = int(sigma_xy[1]*4)
            radius_small_z = int(sigma_z[1]*2)
        else:
            radius_small_xy = radius_small[0]
            radius_small_z = radius_small[1]
        size_small = (radius_small_z*2+1, radius_small_xy*2+1, radius_small_xy*2+1)

        if not radius_large:
            radius_large_xy = int(sigma_xy[1]*30)
            radius_large_xy = int(sigma_z[1]*10)
        else:
            radius_large_xy = radius_large[0]
            radius_large_z = radius_large[1]
        size_large = (radius_large_z*2+1, radius_large_xy*2+1, radius_large_xy*2+1)

        #min_intensity = min_intensity

        gamma = gamma
        normalizer = (sigma_xy[0]**(gamma*2)) * (sigma_z[0]**gamma)

        print(size_large)

(11, 201, 201)


In [6]:
class LinearClassifier2D:
    """
    Sample satisfies `a * X[:,0] + b * X[:,1] + c > 0` is regarded as positive.
    """
    def __init__(self, a,b,c):
        self.a = a
        self.b = b
        self.c = c

    def predict(self, X):
        return self.a * X[:,0] + self.b * X[:,1] + self.c > 0
    
import seaborn as sns
def plot_classification_result(feature_x, feature_y, pred):
    sns.scatterplot(feature_x, feature_y, hue=pred)
    #if np.count_nonzero(pred == 0) > 1:
    #sns.jointplot(x="x", y="y", data=feature_x[pred==0])
    #sns.jointplot(x="x", y="y", data=df)
    sns.kdeplot(feature_x[pred==0],feature_y[pred==0], cmap="Blues", shade=True, shade_lowest=False)
    #if np.count_nonzero(pred == 1) > 1:
    sns.kdeplot(feature_x[pred==1],feature_y[pred==1], cmap="Oranges", shade=True, shade_lowest=False)
    return

In [7]:
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import joblib, json, sys, os
from IPython.display import Image, display_png

sys.path.append("../script/")
import matplotlib.pyplot as plt
import pandas as pd

import sys, os, os.path, re, csv, math
import linecache
import numpy as  np
import pandas as pd
from matplotlib import pyplot as plt
from statistics import mean,stdev
import seaborn as sns

import csv
import pprint

from statannotations.Annotator import Annotator

In [8]:
# Gaussianとmeanを引いたらよいsegmentationができそう。Threshを決める必要はあるが。
# labelを振って、labelごとに重心座標・体積を出すか。

In [8]:
PathPool=["Abeta/#4_APPmodel_Abeta_APP9m_1/"] 
#dir1 = "/export2/Imaging/Axial/"+PathPool[0]
dir1 = "/export3/Imaging/ds4_Data4/"+PathPool[0]
src_dir_SYTOXG = "BOBO-1.tif"
src_dir_Iba1 = "Iba1.tif"
src_dir_Tau = "Abeta.tif"

In [14]:
# 上下さかさまなので、flipする。

src_dir_SYTOXG = "BOBO-1.tif"
src_dir_Iba1 = "Iba1.tif"
src_dir_Tau = "Abeta.tif"

Dir =  "/export3/Imaging/ds4_Data4/"+PathPool[0]+"/" + src_dir_SYTOXG # SYTOXG
image_nuc = tifffile.imread(Dir)
print("OK")

Dir =  "/export3/Imaging/ds4_Data4/"+PathPool[0]+"/" + src_dir_Iba1 # AT8
image_Iba1 = tifffile.imread(Dir)
print("OK")

Dir =  "/export3/Imaging/ds4_Data4/"+PathPool[0]+"/" + src_dir_Tau # AT8
image_tau = tifffile.imread(Dir)
print("OK")

OK
OK
OK


In [15]:
# z , y,  x
image_nuc.shape

(1368, 2304, 2160)

In [16]:
#resize_img = ndimage.zoom(image_nuc, (10/50, 10/50, 10/50), order = 0, mode='nearest')
flip_img = np.flip(image_nuc, axis=1)

dir1 =  "/export3/Imaging/ds4_Data4/"+PathPool[0]

tifffile.imsave(
    dir1+"BOBO-1_flipped.tif",
    flip_img.astype(np.uint16)
)
print("OK")

flip_img = np.flip(image_Iba1, axis=1)

dir1 =  "/export3/Imaging/ds4_Data4/"+PathPool[0]

tifffile.imsave(
    dir1+"Iba1_flipped.tif",
    flip_img.astype(np.uint16)
)
print("OK")

flip_img = np.flip(image_tau, axis=1)

dir1 =  "/export3/Imaging/ds4_Data4/"+PathPool[0]

tifffile.imsave(
    dir1+"Abeta_flipped.tif",
    flip_img.astype(np.uint16)
)
print("OK")


OK
OK
OK


In [17]:
# 上下さかさまをread 
src_dir_SYTOXG = "BOBO-1_flipped.tif"
src_dir_Iba1 = "Iba1_flipped.tif"
src_dir_Tau = "Abeta_flipped.tif"

Dir =  "/export3/Imaging/ds4_Data4/"+PathPool[0]+"/" + src_dir_SYTOXG # SYTOXG
image_nuc = tifffile.imread(Dir)
print("OK")

Dir =  "/export3/Imaging/ds4_Data4/"+PathPool[0]+"/" + src_dir_Iba1 # AT8
image_Iba1 = tifffile.imread(Dir)
print("OK")

Dir =  "/export3/Imaging/ds4_Data4/"+PathPool[0]+"/" + src_dir_Tau # AT8
image_tau = tifffile.imread(Dir)
print("OK")

OK
OK
OK


In [18]:
# 初回だけ 6.5x6.5x8.3um resolutionを 8.3x8.3x8.3に落とす。
# 解像度を落とす場合は、おそらくorder = 0でＯＫ

resize_img = ndimage.zoom(image_nuc, (10/50, 10/50, 10/50), order = 0, mode='nearest')

dir1 =  "/export3/Imaging/ds4_Data4/"+PathPool[0]

tifffile.imsave(
    dir1+"BOBO-1_50um_resized.tif",
    resize_img.astype(np.uint16)
)
print("OK")

resize_img = ndimage.zoom(image_Iba1, (10/50, 10/50, 10/50), order = 0, mode='nearest')

dir1 =  "/export3/Imaging/ds4_Data4/"+PathPool[0]

tifffile.imsave(
    dir1+"Iba1_50um_resized.tif",
    resize_img.astype(np.uint16)
)
print("OK")

resize_img = ndimage.zoom(image_tau, (10/50, 10/50, 10/50), order = 0, mode='nearest')

dir1 =  "/export3/Imaging/ds4_Data4/"+PathPool[0]

tifffile.imsave(
    dir1+"Abeta_50um_resized.tif",
    resize_img.astype(np.uint16)
)
print("OK")

OK
OK
OK


In [19]:
resize_img.shape

(274, 461, 432)

In [21]:
# 上下さかさまをread 
src_dir_SYTOXG = "BOBO-1_flipped.tif"
src_dir_Iba1 = "Iba1_flipped.tif"
src_dir_Tau = "Abeta_flipped.tif"

Dir = "/export3/Imaging/ds4_Data4/"+PathPool[0]+"/" + src_dir_Tau # AT8
image_tau = tifffile.imread(Dir)
print("OK")

OK


In [22]:
from scipy.ndimage import gaussian_filter
from scipy.ndimage import uniform_filter

In [23]:
fsize = 5 #pix四方でmean, gaussianは2*3*sigmaが四方のfilterになる。meanは、2倍の大きさにしておくことでよりならす。
image_tau_gaussed = gaussian_filter(image_tau, sigma=fsize/6)
print("OK")

OK


In [24]:
image_tau_mean = uniform_filter(image_tau, size=fsize*2)
print("OK")

OK


In [25]:
dir1 =  "/export3/Imaging/ds4_Data4/"+PathPool[0]

tifffile.imsave(
    dir1+"image_tau_gaussed.tif",
    image_tau_gaussed.astype(np.uint16)
)
print("OK")

OK


In [26]:
dir1 =  "/export3/Imaging/ds4_Data4/"+PathPool[0]

tifffile.imsave(
    dir1+"image_tau_gaussed1.tif",
    image_tau_gaussed[500].astype(np.uint16)
)
print("OK")

OK


In [27]:
tifffile.imsave(
    dir1+"image_tau_mean.tif",
    image_tau_mean.astype(np.uint16)
)

print("OK")


OK


In [28]:
# 脳の表面は、必ず値が高くなるので、マスクを作製して対処する
# 脳ラベル作製
# 全体のmeanの1.5倍でthreshを引く
mask_thresh = np.average(image_tau) * 1.5
image_tau_brain_mask =  image_tau>mask_thresh
print("OK")

OK


In [29]:
mask_thresh

724.6768644541509

In [30]:
from skimage.filters import threshold_minimum,threshold_mean,threshold_sauvola
#threshold_minimum(image_tau)

In [31]:
image_tau_bg_mask = image_tau<=mask_thresh
print("OK")

OK


In [32]:
tifffile.imsave(
    dir1+"image_tau_brain_mask.tif",
    image_tau_brain_mask.astype(np.uint16)
)
print("OK")

tifffile.imsave(
    dir1+"image_tau_bg_mask.tif",
    image_tau_bg_mask.astype(np.uint16)
)

print("OK")

OK
OK


In [33]:
from scipy import ndimage

In [34]:

Dir = "/export3/Imaging/ds4_Data4/"+PathPool[0]+"/image_tau_brain_mask.tif"
image_tau_brain_mask = tifffile.imread(Dir)

In [35]:
# erosion
image_tau_brain_mask_erosed = ndimage.binary_erosion(image_tau_brain_mask, iterations=10).astype(image_tau_brain_mask.dtype)
print("OK")

OK


In [36]:
dir1 =  "/export3/Imaging/ds4_Data4/"+PathPool[0]
tifffile.imsave(
    dir1+"image_tau_brain_mask_erosed.tif",
    image_tau_brain_mask_erosed.astype(np.uint16)
)

print("OK")

OK


In [37]:
dir1 =  "/export3/Imaging/ds4_Data4/"+PathPool[0]
tifffile.imsave(
    dir1+"image_tau_brain_mask_erosed1.tif",
    image_tau_brain_mask_erosed[500].astype(np.uint16)
)

print("OK")

OK


In [38]:
tifffile.imsave(
    dir1+"image_tau_brain_mask1.tif",
    image_tau_brain_mask[500].astype(np.uint16)
)
print("OK")

OK


In [39]:
Dir =  "/export3/Imaging/ds4_Data4/"+PathPool[0]+"/image_tau_gaussed.tif"
image_tau_gaussed = tifffile.imread(Dir)
Dir =  "/export3/Imaging/ds4_Data4/"+PathPool[0]+"/image_tau_mean.tif"
image_tau_mean = tifffile.imread(Dir)

In [40]:
# gauss- meanで、正のものだけにする
image_tau_diff = np.subtract(image_tau_gaussed.astype(np.float32),image_tau_mean.astype(np.float32))
image_tau_diff = np.where(image_tau_diff>0,  image_tau_diff, 0)
print("OK")

OK


In [41]:
tifffile.imsave(
    dir1+"image_tau_diff.tif",
    image_tau_diff.astype(np.uint16)
)

print("OK")

OK


In [42]:
Dir =  "/export3/Imaging/ds4_Data4/"+PathPool[0]+"/image_tau_diff.tif"
image_tau_diff = tifffile.imread(Dir)

tifffile.imsave(
    dir1+"image_tau_diff_erosed.tif",
    image_tau_diff*image_tau_brain_mask_erosed.astype(np.uint16)
)
print("OK")

OK


In [43]:
tifffile.imsave(
    dir1+"image_tau_diff_erosed1.tif",
    (image_tau_diff*image_tau_brain_mask_erosed)[500].astype(np.uint16)
)
print("OK")

OK


In [44]:
tifffile.imsave(
    dir1+"image_tau_diff1.tif",
    (image_tau_diff)[500].astype(np.uint16)
)
print("OK")

OK


In [45]:
Dir =  "/export3/Imaging/ds4_Data4/"+PathPool[0]+"/image_tau_brain_mask.tif"
image_tau_brain_mask = tifffile.imread(Dir)

In [46]:
Dir =  "/export3/Imaging/ds4_Data4/"+PathPool[0]+"/image_tau_diff_erosed.tif"
image_tau_diff_erosed = tifffile.imread(Dir)

In [47]:
tifffile.imsave(
    dir1+"image_tau_diff_erosed2.tif",
    (image_tau_diff_erosed)[500].astype(np.uint16)
)
print("OK")

OK


In [48]:
Dir =  "/export3/Imaging/ds4_Data4/"+PathPool[0]+"/image_tau_bg_mask.tif"
image_tau_bg_mask = tifffile.imread(Dir)

In [49]:
tifffile.imsave(
    dir1+"image_tau_bg_mask2.tif",
    (image_tau_bg_mask)[500].astype(np.uint16)
)
print("OK")

OK


In [50]:
Dir =  "/export3/Imaging/ds4_Data4/"+PathPool[0]+src_dir_Tau # AT8
image_tau = tifffile.imread(Dir)

In [51]:
# threshold
# bgのノイズの2倍程度で。
# 自動threshも考えたが、微妙。。
# histを自分で出して、きったほうがいいかも？

#thresh = np.average(image_tau, weights=image_tau_bg_mask) 
thresh = np.average(image_tau, weights=image_tau_brain_mask)
thresh = thresh * 1.0
print (thresh)

1423.3861471384487


In [52]:
image_tau_thresh = image_tau_diff_erosed>thresh
#image_tau_thresh = np.where(image_tau_diff>thresh, 1, 0)
print("OK")

OK


In [53]:
tifffile.imsave(
    dir1+"image_tau_thresh.tif",
    image_tau_thresh.astype(np.uint16)
)

print("OK")

OK


In [54]:
tifffile.imsave(
    dir1+"image_tau_thresh1.tif",
    image_tau_thresh[500].astype(np.uint16)
)

print("OK")

OK


In [9]:
Dir =  "/export3/Imaging/ds4_Data4/"+PathPool[0]+"/image_tau_thresh.tif"
image_tau_thresh = tifffile.imread(Dir)

In [10]:
# labelする
image_tau_CCLed=[]
image_tau_CCLed=HDoG_CPU.CCL(image_tau_CCLed, image_tau_thresh)

In [11]:
tifffile.imsave(
    dir1+"image_tau_CCLed1.tif",
    image_tau_CCLed[500].astype(np.float32)
)

print("OK")

OK


In [12]:
tifffile.imsave(
    dir1+"image_tau_CCLed.tif",
    image_tau_CCLed.astype(np.float32)
)

print("OK")

OK


In [ ]:
# labelごとに、重心・体積の集計を行う

In [13]:
Dir =  "/export3/Imaging/ds4_Data4/"+PathPool[0]+"/image_tau_CCLed.tif"
image_tau_CCLed = tifffile.imread(Dir)

Dir =  "/export3/Imaging/ds4_Data4/"+PathPool[0]+"/image_tau_diff.tif"
image_tau_diff = tifffile.imread(Dir)

In [14]:
norm_img = image_tau_diff
labels_img = image_tau_CCLed.astype(np.int32) #labels_img = image_tau_CCLed.astype(np.int64)
on_region = np.nonzero(labels_img)
labels_list = labels_img[on_region]
num_labels = int(np.max(labels_list))
print(num_labels)

182593


In [15]:
# max intensity
max_normalized = []
max_normalized = scipy.ndimage.maximum(norm_img, labels=labels_img, index=range(1, num_labels+1))

# region size
ns = np.ones(len(labels_list))

In [16]:
region_size = np.bincount(labels_list.astype(np.int32)-1, weights=ns)
#region_size = np.bincount(labels_list.astype(np.int64)-1, weights=ns)
#depth=image_tau_CCLed.shape[0]
#width=image_tau_CCLed.shape[1]
#height=image_tau_CCLed.shape[2]

In [17]:
df = pd.DataFrame({
    "index": pd.Series(np.arange(num_labels)),
    "intensity": pd.Series(max_normalized),
    "size": pd.Series(region_size)
})

In [18]:
df

,index,intensity,size
0,0,4927,11.0
1,1,4108,118.0
2,2,5072,157.0
3,3,4629,110.0
4,4,2910,35.0
...,...,...,...
182588,182588,21569,199.0
182589,182589,1853,16.0
182590,182590,1638,11.0
182591,182591,1499,2.0


In [19]:
dir1 =  "/export3/Imaging/ds4_Data4/"+PathPool[0]

csv_tau = dir1 + "csv_tau1.csv"

df= df.astype('int')
df.to_csv(csv_tau, index=False, header=True, chunksize=50000,
            columns=["index","intensity","size"])

In [20]:
from scipy.ndimage.measurements import center_of_mass

centroids = center_of_mass(image_tau_diff, labels=labels_img, index=range(1, num_labels+1))
centroid_z, centroid_y, centroid_x = zip(*centroids)

In [21]:
Dir = "/export3/Imaging/ds4_Data4/"+PathPool[0]+"/image_tau_CCLed.tif"
labels_img = tifffile.imread(Dir).astype(np.int32) #labels_img = tifffile.imread(Dir).astype(np.int64)

In [22]:
labels_img.shape

(1368, 2304, 2160)

In [23]:
dir1 =  "/export3/Imaging/ds4_Data4/"+PathPool[0]
data_1 = pd.read_csv(filepath_or_buffer=dir1 + "csv_tau1.csv", encoding="ms932", sep=",")

In [24]:
data_1

,index,intensity,size
0,0,4927,11
1,1,4108,118
2,2,5072,157
3,3,4629,110
4,4,2910,35
...,...,...,...
182588,182588,21569,199
182589,182589,1853,16
182590,182590,1638,11
182591,182591,1499,2


In [25]:
index1=np.array(data_1["index"])

In [26]:
df = pd.DataFrame({
    "index":  index1,
    "centroid_x": pd.Series(centroid_x),
    "centroid_y": pd.Series(centroid_y),
    "centroid_z": pd.Series(centroid_z),
})

In [27]:
df

,index,centroid_x,centroid_y,centroid_z
0,0,1354.831582,1216.863271,65.150221
1,1,1314.356871,1236.062856,68.231632
2,2,1301.592156,1122.616467,69.712328
3,3,1262.838835,1173.762954,69.492205
4,4,1374.184116,1199.055714,66.795161
...,...,...,...,...
182588,182588,1096.304892,2014.810917,1279.005311
182589,182589,1167.369510,2082.431977,1275.652717
182590,182590,1159.355720,2108.363474,1282.173267
182591,182591,1173.000000,2037.000000,1284.489615


In [28]:
dir1 =  "/export3/Imaging/ds4_Data4/"+PathPool[0]

csv_tau = dir1 + "csv_tau2.csv"

df= df.astype('int')
df.to_csv(csv_tau, index=False, header=True, chunksize=50000,
            columns=["index","centroid_x","centroid_y","centroid_z"])

In [29]:
# umに直す

In [30]:
data_1 = pd.read_csv(filepath_or_buffer=dir1 + "csv_tau1.csv", encoding="ms932", sep=",")
data_2 = pd.read_csv(filepath_or_buffer=dir1 + "csv_tau2.csv", encoding="ms932", sep=",")

In [31]:
data_2

,index,centroid_x,centroid_y,centroid_z
0,0,1354,1216,65
1,1,1314,1236,68
2,2,1301,1122,69
3,3,1262,1173,69
4,4,1374,1199,66
...,...,...,...,...
182588,182588,1096,2014,1279
182589,182589,1167,2082,1275
182590,182590,1159,2108,1282
182591,182591,1173,2037,1284


In [32]:
blunk_array_0 =np.full(len(data_2["centroid_x"]), 0.0)

In [33]:
df12 = []
df12 = pd.DataFrame({
    "X":pd.Series(data_2["centroid_x"], dtype=np.int64),
    "Y":pd.Series(data_2["centroid_y"], dtype=np.int64),
    "Z":pd.Series(data_2["centroid_z"], dtype=np.int64),
    "deltaI":pd.Series(data_1["intensity"], dtype=np.int64),
    "BG":pd.Series(blunk_array_0, dtype=np.int64),
    "vol":pd.Series(data_1["size"], dtype=np.int64),
    "deltaI_total":pd.Series(data_1["intensity"], dtype=np.int64)
        })

In [34]:
df12

,X,Y,Z,deltaI,BG,vol,deltaI_total
0,1354,1216,65,4927,0,11,4927
1,1314,1236,68,4108,0,118,4108
2,1301,1122,69,5072,0,157,5072
3,1262,1173,69,4629,0,110,4629
4,1374,1199,66,2910,0,35,2910
...,...,...,...,...,...,...,...
182588,1096,2014,1279,21569,0,199,21569
182589,1167,2082,1275,1853,0,16,1853
182590,1159,2108,1282,1638,0,11,1638
182591,1173,2037,1284,1499,0,2,1499


In [35]:
df12["X"].max()

1950

In [36]:
df12["Y"].max()

2212

In [37]:
df12["Z"].max()

1289

In [38]:
dir1 = "/export3/Imaging/ds4_Data4/"+PathPool[0]

csv_tau = dir1 + "csv_tau12.csv"

df12= df12.astype('int')
df12.to_csv(csv_tau, index=False, header=True, chunksize=50000,
            columns=["X","Y","Z","deltaI","BG","vol","deltaI_total"])

In [39]:
# um解像度
scale = 10 #um

df13 = []
df13 = pd.DataFrame({
    "X":pd.Series(data_2["centroid_x"]*scale, dtype=np.int64),
    "Y":pd.Series(data_2["centroid_y"]*scale, dtype=np.int64),
    "Z":pd.Series(data_2["centroid_z"]*scale, dtype=np.int64),
    "deltaI":pd.Series(data_1["intensity"], dtype=np.int64),
    "BG":pd.Series(blunk_array_0, dtype=np.int64),
    "vol":pd.Series(data_1["size"]*scale*scale*scale, dtype=np.int64),
    "deltaI_total":pd.Series(data_1["intensity"], dtype=np.int64)
        })


dir1 =  "/export3/Imaging/ds4_Data4/"+PathPool[0]

csv_tau = dir1 + "csv_tau12_um.csv"

df13= df13.astype('int')
df13.to_csv(csv_tau, index=False, header=True, chunksize=50000,
            columns=["X","Y","Z","deltaI","BG","vol","deltaI_total"])

In [40]:
df13

,X,Y,Z,deltaI,BG,vol,deltaI_total
0,13540,12160,650,4927,0,11000,4927
1,13140,12360,680,4108,0,118000,4108
2,13010,11220,690,5072,0,157000,5072
3,12620,11730,690,4629,0,110000,4629
4,13740,11990,660,2910,0,35000,2910
...,...,...,...,...,...,...,...
182588,10960,20140,12790,21569,0,199000,21569
182589,11670,20820,12750,1853,0,16000,1853
182590,11590,21080,12820,1638,0,11000,1638
182591,11730,20370,12840,1499,0,2000,1499


In [41]:
# いったん画像で確認
scale=10 # um
scale=scale/50 # 50um scaleに変換

ptsw25 = []
ptsw25 = pd.DataFrame({
    "x":pd.Series(np.array(df12["X"])*scale, dtype=np.float),
    "y":pd.Series(np.array(df12["Y"])*scale, dtype=np.float),
    "z":pd.Series(np.array(df12["Z"])*scale, dtype=np.float),
        })

In [42]:
ptsw25

,x,y,z
0,270.8,243.2,13.0
1,262.8,247.2,13.6
2,260.2,224.4,13.8
3,252.4,234.6,13.8
4,274.8,239.8,13.2
...,...,...,...
182588,219.2,402.8,255.8
182589,233.4,416.4,255.0
182590,231.8,421.6,256.4
182591,234.6,407.4,256.8


In [43]:
#depth_ori = int(np.floor(np.max(ptsw25['z'].values)))
#height_ori =int(np.floor(np.max(ptsw25['y'].values)))
#width_ori = int(np.floor(np.max(ptsw25['x'].values)))
# (1068, 2005, 1692)
# (1245, 2304, 1930)
depth_ori = int(1245*scale)
height_ori =int(2304*scale)
width_ori = int(1930*scale)

img_N_ori,_ = np.histogramdd(
    np.vstack([
        ptsw25['z'].values,
        ptsw25['y'].values,
        ptsw25['x'].values
        ]).T,
        bins=(depth_ori, height_ori, width_ori),
        range=[(0,depth_ori-1),(0,height_ori-1),(0,width_ori-1)]
    )
        

In [44]:
tifffile.imsave(
        dir1+ "tau_density_50um.tif",
        img_N_ori.astype(np.uint16)
    )

In [45]:
# CUBIC-Cloudに投げる

In [46]:
# 必要あればさらに閾値を切ることはできる。
#classify_unsupervised(self, features, i_feature_maximize=2, n_components=3):